In [ ]:
from dotenv import load_dotenv
load_dotenv()

: 

In [ ]:
import os
from langchain.agents import tool
from crewai_tools import PDFSearchTool
from crewai import Agent, Crew, Task,Process
from langchain_community.tools.tavily_search import TavilySearchResults
from langchain_openai import ChatOpenAI

In [ ]:
import langchain
import crewai
print(langchain.__version__)
import langchain_community
print(langchain_community.__version__)


In [ ]:

# Use GROQ_API_KEY
llm =ChatOpenAI(
    openai_api_key=os.environ['GROQ_API_KEY'],
    openai_api_base="https://api.groq.com/openai/v1",
    model_name="llama3-8b-8192",
    temperature=0.1,
    max_tokens=1000,

)

"""
# OR Use Ollama
from langchain_ollama import OllamaLLM

llm = OllamaLLM(model="llama3.2:latest")

llm.invoke("The first man on the moon was ...")

"""


### Custom model and embeddings

In [ ]:
# default the tool uses OpenAI for both embeddings and summarization
pdf_tool = PDFSearchTool(pdf='attentions_is_all_you_need.pdf',
    config=dict(
        llm=dict(
            provider="groq", # or google, openai, anthropic, llama2, ...
            config=dict(
                model="llama3-8b-8192",
                # temperature=0.5,
                # top_p=1,
                # stream=true,
            ),
        ),
        embedder=dict(
            provider="huggingface", # or openai, ollama, ...
            config=dict(
                model="BAAI/bge-small-en-v1.5",
                #task_type="retrieval_document",
                # title="Embeddings",
            ),
        ),
    )
)
# Web Search Tool
web_search_tool = TavilySearchResults()

### test PDF tools

In [ ]:
# test pdf tools
question  = "How did self-attention mechanism evolve in large language models?"
print(pdf_tool.run(question))

In [ ]:
# Test Web search Tools
question  = "How did self-attention mechanism evolve in large language models?"
web_search_tool.run(question)

In [ ]:
# Define Router Tool
@tool
def simple_router_tool(question: str) -> str:
    """Routes queries to the appropriate resource."""
    if "pdf" in question.lower():
        return "pdf"
    else:
        return "web"


In [ ]:
question  = "How did self-attention mechanism evolve in large language models in pdf?"
simple_router_tool.run(question)


## Agents Creation

In [ ]:
# Router Agent
router_agent = Agent(
    role="Router",
    goal="Determine if the query should go to the PDF or Web search.",
    backstory=(
        "You analyze user queries to decide whether to search a document "
        "or perform a general web search."
    ),
    llm=llm, #default ChatGPT
)


router_task = Task(
    description=(
        "Analyze the user question '{question}'. "
        "If the question involves 'document', route it to the PDF search. "
        "Otherwise, route it to web search."
    ),
    expected_output="Return 'pdf' for PDF search or 'web' for web search.",
    agent=router_agent,
    tools=[simple_router_tool],
)

In [ ]:
rag_crew1 = Crew(
    agents=[router_agent],
    tasks=[router_task],
    verbose=True,

)

test_input = {"question": "What does the  say about machine learning?"}
result = rag_crew1.kickoff(inputs=test_input)

In [ ]:
print(result)

In [ ]:
# Retriever Agent
retriever_agent = Agent(
    role="Retriever",
    goal="Retrieve information from either a PDF or the web.",
    backstory=(
        "You are responsible for retrieving information from the appropriate "
        "resource (PDF or web) based on the router's decision."
    ),
    llm=llm,
)

retriever_task = Task(
    description=(
        "Based on the routing decision ('pdf' or 'web'), use the relevant tool to retrieve information. "
        "If 'pdf', use the PDF search tool. If 'web', use the web search tool."
    ),
    expected_output="Provide a clear, concise answer using the retrieved information.",
    context=[router_task],
    tools=[pdf_tool, web_search_tool],
    agent=retriever_agent,
)

In [ ]:
# Crew for the System
simple_rag_crew = Crew(
    agents=[router_agent, retriever_agent],
    tasks=[router_task, retriever_task],
    verbose=True,
)

In [ ]:
# Example Input
inputs = {"question": "What does the say about machine learning?"}

#inputs ={"question":"How does self-attention mechanism help large language models?"}
# Run the System
response = simple_rag_crew.kickoff(inputs)
print(response)

In [ ]:
print(response)